Podríamos usar este modelo para la clasificación: [BioBERT](https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT)

Clono repositorio para obtener mi datos y organizarlos

In [2]:
!git clone https://github.com/Vicomtech/NUBes-negation-uncertainty-biomedical-corpus.git

fatal: destination path 'NUBes-negation-uncertainty-biomedical-corpus' already exists and is not an empty directory.


Instalo las dependencias que se van a utilizar

In [3]:
%pip install openai -qqq argilla -qqq sentence-transformers==2.2.2 -qqq setfit==0.6.0 -qqq datasets ir-datasets

In [4]:
import os
from json import loads

import openai

from datasets import load_dataset
from transformers import AutoTokenizer

import pandas as pd

from argilla.metrics.text_classification import f1
from argilla.metrics.commons import text_length

from setfit import get_templated_dataset
from setfit import SetFitModel, SetFitTrainer

from sentence_transformers import SentenceTransformer
from sentence_transformers.losses import CosineSimilarityLoss

In [5]:
import argilla as rg

In [6]:
rg.init(
    api_url="https://dylanmontoya22-ner-biomedical-text.hf.space",
    api_key="team.apikey"
)

In [7]:
data_files = {
    'train': '/content/NUBes-negation-uncertainty-biomedical-corpus/IULA+/negation_iac_1_corr.nubes.txt'
}

dataset = load_dataset('text', data_files=data_files)

  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
dataset['train']['text'][0:10]

['Sutura 2.0 y cura tópica con povidona yodada.',
 'Anodino.',
 'Ictericia cutáneo-mucosa.',
 'Sin contraindicaciones para el procedimiento.',
 'Hipertenso.',
 '-',
 '- Laboratorio',
 'Sedación farmacológica.',
 'Pupilas isocóricas normorreactivas.',
 '-']

In [9]:
%pip install "flair~=0.11.0" -qqq

In [10]:
from flair.models import TARSTagger
from flair.data import Sentence

In [11]:
# define labels
labels = ["signosintoma", "procedimiento", "anatomia", "problemaclinico"]

# load zero-shot NER tagger
tars = TARSTagger.load("tars-ner")

# define labels for named entities using wnut labels
tars.add_and_switch_to_new_task("task 1", labels, label_type="ner")

2023-03-26 04:52:27,124 loading file /root/.flair/models/tars-ner.pt
2023-03-26 04:52:47,309 SequenceTagger predicts: Dictionary with 5 tags: O, S-entity, B-entity, E-entity, I-entity


In [12]:
# wrap our tokens in a flair Sentence
sentence = Sentence(" ".join(dataset['train']['text']))

# add predictions to our sentence
tars.predict(sentence)

# extract predicted entities into a list of tuples (entity, start_char, end_char)
[
    (entity.get_labels()[0].value, entity.start_position, entity.end_position)
    for entity in sentence.get_spans("ner")
]

[]

In [17]:
# build records for the first 100 examples
records = []

for record in dataset['train'].select(range(100)):
    input_text = " ".join(record['text'].split())

    sentence = Sentence(input_text)
    tars.predict(sentence)
    prediction = [
        (entity.get_labels()[0].value, entity.start_position, entity.end_position)
        for entity in sentence.get_spans("ner")
    ]

    # building TokenClassificationRecord
    records.append(
        rg.TokenClassificationRecord(
            text=input_text,
            tokens=[token.text for token in sentence],
            prediction=prediction,
            prediction_agent="tars-ner",
        )
    )

# log the records to Argilla
rg.log(records, name="test_func_tokens_3", metadata={"split": "test"})

Output()

100 records logged to ]8;id=704777;https://dylanmontoya22-ner-biomedical-text.hf.space/datasets/team/test_func_tokens_3\https://dylanmontoya22-ner-biomedical-text.hf.space/datasets/team/test_func_tokens_3]8;;\

BulkResponse(dataset='test_func_tokens_3', processed=100, failed=0)